In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!unzip -q /kaggle/input/aerial-cactus-identification/train.zip
!unzip -q /kaggle/input/aerial-cactus-identification/test.zip

In [ ]:
# from PIL import Image

# Image.open("train/000c8a36845c0208e833c79c1bffedd1.jpg")

In [ ]:
import glob

train_path = glob.glob("train/*")
test_path = glob.glob("test/*")

In [ ]:
train = pd.read_csv("/kaggle/input/aerial-cactus-identification/train.csv")
train.columns = ["path","has_cactus"]
train["path"] = "train/"+train["path"]
train["has_cactus"] = train["has_cactus"].astype(str)
train

In [ ]:
test = pd.DataFrame({"path":test_path})
test

In [ ]:
!pip install -U git+https://github.com/qubvel/efficientnet

In [ ]:
from keras import Sequential
from keras.layers import *
from efficientnet.tfkeras import EfficientNetB0 
from keras.callbacks import ReduceLROnPlateau,EarlyStopping,ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import StratifiedKFold

sfk = StratifiedKFold(n_splits = 5, random_state = 42, shuffle = True)

idg2 = ImageDataGenerator()
test_generator = idg2.flow_from_dataframe(test, x_col = "path", y_col = None, batch_size= 128, target_size=(200,200), class_mode = None, shuffle = False)

result = 0

for i,j in sfk.split(train,train["has_cactus"]):
    
    x_train = train.iloc[i]
    x_valid = train.iloc[j]

    idg = ImageDataGenerator(horizontal_flip = True)
    train_generator = idg.flow_from_dataframe(x_train, x_col = "path" , y_col = "has_cactus", batch_size = 128, target_size = (200,200))
    valid_generator = idg.flow_from_dataframe(x_valid, x_col = "path" , y_col = "has_cactus", batch_size = 128, target_size = (200,200))
    
    rl = ReduceLROnPlateau(patience = 2)
    es = EarlyStopping(patience = 3, verbose = 1)
    mc = ModelCheckpoint("best.h5", save_best_only = True, verbose = 1 )

    model = Sequential()

    model.add(EfficientNetB0(weights = "imagenet", include_top = False, pooling = "avg"))

    model.add(Dense(2, activation = "softmax"))

    model.compile(metrics = ["acc"], loss = "categorical_crossentropy", optimizer = "adam")

    model.fit(train_generator, validation_data = valid_generator, epochs = 10, callbacks = [es,mc,rl])

    model.load_weights("best.h5")

    result += model.predict(test_generator)/5



In [ ]:
sub = pd.read_csv("/kaggle/input/aerial-cactus-identification/sample_submission.csv")
sub.head()

In [ ]:
sub["id"] = test_generator.filenames
sub["id"] = sub["id"].apply(lambda x : x.split("/")[1])

sub["has_cactus"] = result[:,1]
sub.head()

In [ ]:
sub.to_csv("submission.csv", index = False)